# Fitted Q-Learning: Function Approximation for Continuous State Spaces

In the previous TP, we studied tabular Q-Learning for discrete MDPs. This method stores a separate Q-value for each state-action pair in a table. However, this approach fails when:

1. The state space is continuous (e.g., robot positions, velocities)
2. The state space is too large to enumerate (e.g., image observations)

Fitted Q-Learning solves this problem using **function approximation**: instead of storing Q-values in a table, we learn a function $\hat{Q}(s, a; \mathbf{w})$ parameterized by weights $\mathbf{w}$.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures

## 1. Function Approximation

### 1.1 Linear Q-function approximation

We approximate the Q-function as a linear combination of features:

$\hat{Q}(s, a; \mathbf{w}) = \phi(s, a)^T \mathbf{w}$

where:
- $\phi(s, a)$ is a feature vector extracted from state $s$ and action $a$
- $\mathbf{w}$ are learnable weights

### 1.2 Fitted Q-Iteration (FQI)

FQI is a batch reinforcement learning algorithm that learns Q-values from a dataset of transitions $\mathcal{D} = \{(s_i, a_i, r_i, s'_i)\}_{i=1}^N$.

The algorithm iteratively refines the Q-function approximation:

1. Initialize $\hat{Q}^{(0)}$
2. For iteration $k = 1, 2, \ldots, K$:
   - For each transition $(s_i, a_i, r_i, s'_i)$ in $\mathcal{D}$:
     - Compute target: $y_i = r_i + \gamma \max_{a'} \hat{Q}^{(k-1)}(s'_i, a')$
   - Update: $\hat{Q}^{(k)} = \underset{\hat{Q}}{\operatorname{argmin}} \sum_i (\hat{Q}(s_i, a_i) - y_i)^2$

This converts the RL problem into a sequence of supervised regression problems.

## 2. Continuous GridWorld Environment

We consider a continuous version of the GridWorld:

- **State space**: $s = (x, y) \in [0, \text{grid\_size}]^2$ (continuous)
- **Action space**: $a \in \{0, 1, 2, 3\}$ = {up, down, left, right} (discrete)
- **Dynamics**: The agent moves $\delta = 0.5$ in the chosen direction, with Gaussian noise $\mathcal{N}(0, 0.1^2)$ added to both coordinates
- **Rewards**:
  - Reaching the goal (distance < 0.3): $+10$ (terminal)
  - Each step: $-0.1$
- **Initial state**: Near bottom-left corner
- **Goal**: Top-right corner

### 2.1 The Continuous GridWorld environment

We provide a complete implementation of the continuous GridWorld. Students will use this environment to test Fitted Q-Learning.

In [ ]:
class ContinuousGridWorld:
    def __init__(self, grid_size=5.0, goal_pos=None, noise_std=1.0, step_size=0.5):
        self.grid_size = grid_size
        self.goal_pos = goal_pos if goal_pos is not None else np.array([grid_size - 0.5, grid_size - 0.5])
        self.noise_std = noise_std
        self.step_size = step_size
        self.goal_radius = 0.3
        
        # Action mapping: 0=up, 1=down, 2=left, 3=right
        self.action_effects = {
            0: np.array([0, self.step_size]),   # up
            1: np.array([0, -self.step_size]),  # down
            2: np.array([-self.step_size, 0]),  # left
            3: np.array([self.step_size, 0])    # right
        }
        
        self.current_state = None
        self.timestep = 0
        self.max_steps = 200
    
    def reset(self):
        """Initialize episode with random start position near bottom-left"""
        self.current_state = np.array([0.5, 0.5]) + np.random.randn(2) * 0.2
        self.current_state = np.clip(self.current_state, 0, self.grid_size)
        self.timestep = 0
        return self.current_state.copy()
    
    def step(self, action):
        """Execute action and return next state, reward, done"""
        # Apply action with Gaussian noise
        noise = np.random.randn(2) * self.noise_std
        next_state = self.current_state + self.action_effects[action] + noise
        
        # Clip to grid boundaries
        next_state = np.clip(next_state, 0, self.grid_size)
        
        # Compute reward and check if done
        distance_to_goal = np.linalg.norm(next_state - self.goal_pos)
        
        if distance_to_goal < self.goal_radius:
            reward = 10.0
            done = True
        elif self.timestep >= self.max_steps:
            reward = -0.1
            done = True
        else:
            reward = -0.1
            done = False
        
        self.current_state = next_state
        self.timestep += 1
        
        return next_state.copy(), reward, done
    
    def render(self, trajectories=None):
        """Visualize the environment and optional trajectories"""
        plt.figure(figsize=(6, 6))
        
        # Draw goal
        circle = plt.Circle(self.goal_pos, self.goal_radius, color='green', alpha=0.3, label='Goal')
        plt.gca().add_patch(circle)
        plt.plot(self.goal_pos[0], self.goal_pos[1], 'g*', markersize=15)
        
        # Draw trajectories if provided
        if trajectories is not None:
            for traj in trajectories:
                states = np.array(traj)
                plt.plot(states[:, 0], states[:, 1], 'b-', alpha=0.3, linewidth=0.5)
                plt.plot(states[0, 0], states[0, 1], 'ro', markersize=5)
        
        plt.xlim(-0.2, self.grid_size + 0.2)
        plt.ylim(-0.2, self.grid_size + 0.2)
        plt.xlabel('x')
        plt.ylabel('y')
        plt.title('Continuous GridWorld')
        plt.grid(True, alpha=0.3)
        plt.legend()
        plt.axis('equal')
        plt.show()

Test the environment with a random policy:

In [ ]:
env = ContinuousGridWorld()

# Collect a few random trajectories
trajectories = []
for _ in range(5):
    traj = []
    state = env.reset()
    traj.append(state)
    done = False
    
    while not done:
        action = np.random.randint(4)
        state, reward, done = env.step(action)
        traj.append(state)
    
    trajectories.append(traj)

env.render(trajectories)

### 2.2 Why tabular Q-Learning may fails

To understand why we need function approximation, let's try discretizing the continuous state space and analyze the coverage.

In [ ]:
def discretize_state(state, grid_size, n_bins=10):
    """Discretize continuous state into bins"""
    # TODO: Compute bin indices for state
    raise NotImplementedError
    return bin_x, bin_y

# Collect data with random policy
n_bins = 10
visit_counts = np.zeros((n_bins, n_bins))

for episode in range(50):
    state = env.reset()
    done = False
    
    while not done:
        bin_x, bin_y = discretize_state(state, env.grid_size, n_bins)
        visit_counts[bin_x, bin_y] += 1
        
        action = np.random.randint(4)
        state, reward, done = env.step(action)

# Visualize state coverage
plt.figure(figsize=(8, 6))
plt.imshow(visit_counts.T, origin='lower', cmap='Blues')
plt.colorbar(label='Visit count')
plt.xlabel('x bin')
plt.ylabel('y bin')
plt.title(f'State visitation with {n_bins}x{n_bins} discretization')
plt.show()

# Print statistics
total_bins = n_bins * n_bins
visited_bins = np.sum(visit_counts > 0)
print(f"Visited {visited_bins}/{total_bins} bins ({100*visited_bins/total_bins:.1f}%)")
print(f"Many bins are never visited, so tabular Q-learning would have no data to learn from!")

## 3. Data Collection

Fitted Q-Iteration is a batch RL algorithm: it learns from a fixed dataset of transitions.

### 3.1 Collect a dataset of transitions

Collect transitions $(s, a, r, s', \text{done})$ by running episodes with a random policy.

In [ ]:
def collect_data(env, num_episodes=100):
    """Collect transitions using random policy"""
    dataset = []
    np.random.seed(1)
    
    for episode in range(num_episodes):
        state = env.reset()
        done = False
        
        while not done:
            action = np.random.randint(4)
            next_state, reward, done = env.step(action)
            
            dataset.append((state, action, reward, next_state, done))
            state = next_state
    
    return dataset

# Collect data
dataset = collect_data(env, num_episodes=50)
print(f"Collected {len(dataset)} transitions")

Visualize the collected data:

In [ ]:
# Extract states and rewards
states = np.array([transition[0] for transition in dataset])
rewards = np.array([transition[2] for transition in dataset])

# Plot state distribution
plt.figure(figsize=(12, 4))

plt.subplot(1, 3, 1)
plt.scatter(states[:, 0], states[:, 1], alpha=0.1, s=1)
plt.xlabel('x')
plt.ylabel('y')
plt.title('Visited states')
plt.grid(True, alpha=0.3)

plt.subplot(1, 3, 2)
actions = [transition[1] for transition in dataset]
plt.hist(actions, bins=4, range=(-0.5, 3.5), edgecolor='black')
plt.xlabel('Action')
plt.ylabel('Count')
plt.title('Action distribution')
plt.xticks([0, 1, 2, 3], ['up', 'down', 'left', 'right'])

plt.subplot(1, 3, 3)
plt.hist(rewards, bins=20, edgecolor='black')
plt.xlabel('Reward')
plt.ylabel('Count')
plt.title('Reward distribution')

plt.tight_layout()
plt.show()

## 4. Feature Engineering

To use linear regression, we need to convert $(s, a)$ pairs into feature vectors $\phi(s, a)$.

### 4.1 Polynomial features for states

For a 2D state $s = (x, y)$, polynomial features of degree 2 are:

$\phi_{\text{state}}(s) = [1, x, y, x^2, xy, y^2]$

For degree 3: $[1, x, y, x^2, xy, y^2, x^3, x^2y, xy^2, y^3]$

### 4.2 Action encoding

We combine state features with one-hot encoded actions. For 4 actions and degree 2 state features (6 dimensions), we get:

$\phi(s, a) = [\phi_{\text{state}}(s) \text{ if } a=0 \text{ else } \mathbf{0}, \ldots, \phi_{\text{state}}(s) \text{ if } a=3 \text{ else } \mathbf{0}]$

This creates a $6 \times 4 = 24$ dimensional feature vector.

### 4.3 Exercise : Implement feature extraction

In [ ]:
class FeatureExtractor:
    def __init__(self, degree=2, n_actions=4):
        self.degree = degree
        self.n_actions = n_actions
        self.poly = PolynomialFeatures(degree=degree)
        
        # Fit on dummy data to initialize
        dummy = np.array([[0, 0]])
        self.poly.fit(dummy)
        self.n_state_features = self.poly.transform(dummy).shape[1]
    
    def extract(self, state, action):
        """Extract feature vector for (state, action) pair"""
        # TODO: Compute polynomial features for state
        # TODO: Create feature vector with state features at action index
        # Hint: Create vector of size (n_state_features * n_actions)
        raise NotImplementedError
    
    def extract_batch(self, states, actions):
        """Extract features for batch of (state, action) pairs"""
        return np.array([self.extract(s, a) for s, a in zip(states, actions)])

Test the feature extractor:

In [ ]:
feat = FeatureExtractor(degree=2)
test_state = np.array([1.0, 2.0])
test_action = 0
features = feat.extract(test_state, test_action)
print(f"Feature vector shape: {features.shape}")
print(f"Number of features: {len(features)} (= {feat.n_state_features} state features x {feat.n_actions} actions)")

## 5. Fitted Q-Iteration

### 5.1 Exercise : Implement the FQI algorithm

The algorithm:

1. Extract features for all $(s, a)$ pairs in the dataset
2. Initialize Q-function (linear regression model)
3. For each iteration:
   - Compute target values: $y_i = r_i + \gamma \max_{a'} \hat{Q}(s'_i, a')$
   - Fit Q-function to predict targets from features
4. Return learned Q-function

In [ ]:
class QFunction:
    def __init__(self, feature_extractor):
        self.feat = feature_extractor
        self.model = LinearRegression()
        self.is_fitted = False
    
    def predict(self, state, action):
        """Predict Q-value for (state, action)"""
        if not self.is_fitted:
            return 0.0
        features = self.feat.extract(state, action).reshape(1, -1)
        return self.model.predict(features)[0]
    
    def predict_batch(self, states, actions):
        """Predict Q-values for batch"""
        if not self.is_fitted:
            return np.zeros(len(states))
        features = self.feat.extract_batch(states, actions)
        return self.model.predict(features)
    
    def max_q(self, state):
        """Compute max_a Q(state, a)"""
        q_values = [self.predict(state, a) for a in range(self.feat.n_actions)]
        return np.max(q_values)
    
    def greedy_action(self, state):
        """Return argmax_a Q(state, a)"""
        q_values = [self.predict(state, a) for a in range(self.feat.n_actions)]
        return np.argmax(q_values)
    
    def fit(self, features, targets):
        """Fit Q-function to targets"""
        self.model.fit(features, targets)
        self.is_fitted = True

def fitted_q_iteration(dataset, degree=2, gamma=0.95, num_iterations=20):
    """Fitted Q-Iteration algorithm"""
    # Initialize feature extractor and Q-function
    feat = FeatureExtractor(degree=degree)
    q_func = QFunction(feat)
    
    # Extract components from dataset
    states = np.array([t[0] for t in dataset])
    actions = np.array([t[1] for t in dataset])
    rewards = np.array([t[2] for t in dataset])
    next_states = np.array([t[3] for t in dataset])
    dones = np.array([t[4] for t in dataset])
    
    # Extract features for all (s, a) pairs
    features = feat.extract_batch(states, actions)
    
    # FQI iterations
    for iteration in range(num_iterations):
        # TODO: Compute targets for each transition
        targets = None
        
        # TODO: Fit Q-function
        raise NotImplementedError
        
        if iteration % 5 == 0:
            mean_target = np.mean(targets)
            print(f"Iteration {iteration}: mean target = {mean_target:.3f}")
    
    return q_func

Run Fitted Q-Iteration:

In [ ]:
q_func = fitted_q_iteration(dataset, degree=3, gamma=0.95, num_iterations=20)

## 6. Policy Evaluation

### 6.1 Exercise : Extract and evaluate the learned policy

In [ ]:
def evaluate_policy(env, policy_fn, num_episodes=50, render_episodes=0):
    """Evaluate policy and return statistics"""
    scores = []
    successes = 0
    trajectories = []
    
    for episode in range(num_episodes):
        state = env.reset()
        done = False
        episode_reward = 0
        traj = [state.copy()]
        
        # TODO: Run one episode following policy_fn
        # - Use policy_fn(state) to select actions
        # - Accumulate rewards in episode_reward
        # - Store states in traj
        raise NotImplementedError
        
        scores.append(episode_reward)
        if reward > 5:  # Reached goal
            successes += 1
        
        if episode < render_episodes:
            trajectories.append(traj)
    
    return {
        'mean_score': np.mean(scores),
        'std_score': np.std(scores),
        'success_rate': successes / num_episodes,
        'trajectories': trajectories
    }

# Evaluate learned policy
learned_policy = lambda s: q_func.greedy_action(s)
results_learned = evaluate_policy(env, learned_policy, num_episodes=50, render_episodes=3)

# Compare with random policy
random_policy = lambda s: np.random.randint(4)
results_random = evaluate_policy(env, random_policy, num_episodes=50)

print("Learned policy:")
print(f"  Mean score: {results_learned['mean_score']:.2f} +/- {results_learned['std_score']:.2f}")
print(f"  Success rate: {results_learned['success_rate']:.1%}")

print("\nRandom policy:")
print(f"  Mean score: {results_random['mean_score']:.2f} +/- {results_random['std_score']:.2f}")
print(f"  Success rate: {results_random['success_rate']:.1%}")

Visualize learned policy trajectories:

In [ ]:
env.render(results_learned['trajectories'])

### 6.2 Exercise : Visualize the learned Q-function

Plot heatmaps of $\hat{Q}(s, a)$ for each action.

In [ ]:
def visualize_q_function(q_func, grid_size=5.0, resolution=50):
    """Plot Q-function heatmaps for each action"""
    # Create grid of states
    x = np.linspace(0, grid_size, resolution)
    y = np.linspace(0, grid_size, resolution)
    X, Y = np.meshgrid(x, y)
    
    action_names = ['Up', 'Down', 'Left', 'Right']
    
    fig, axes = plt.subplots(2, 2, figsize=(12, 10))
    axes = axes.flatten()
    
    # TODO: For each action (0-3):
    # 1. Create Q_values array of shape (resolution, resolution)
    # 2. For each grid position (i, j):
    #    - Create state from X[i, j], Y[i, j]
    #    - Compute Q_values[i, j]
    # 3. Plot using axes[action].imshow() with extent=[0, grid_size, 0, grid_size]
    # 4. To mark the goal you can use axes[action].plot(env.goal_pos[0], env.goal_pos[1], 'r*', markersize=15)
    raise NotImplementedError
    
    plt.tight_layout()
    plt.show()

visualize_q_function(q_func)

Visualize the policy:

In [ ]:
def visualize_policy(q_func, grid_size=5.0, resolution=100):
    """Plot policy as color-coded heatmap"""
    x = np.linspace(0, grid_size, resolution)
    y = np.linspace(0, grid_size, resolution)
    X, Y = np.meshgrid(x, y)
    
    # Compute best action at each grid point
    policy_map = np.zeros((resolution, resolution))
    for i in range(resolution):
        for j in range(resolution):
            state = np.array([X[i, j], Y[i, j]])
            policy_map[i, j] = q_func.greedy_action(state)
    
    # Color map: 0=up (blue), 1=down (orange), 2=left (green), 3=right (red)
    plt.figure(figsize=(8, 7))
    im = plt.imshow(policy_map, origin='lower', extent=[0, grid_size, 0, grid_size],
                    cmap='tab10', vmin=0, vmax=3, aspect='auto')
    
    # Add colorbar with action labels
    cbar = plt.colorbar(im, ticks=[0, 1, 2, 3])
    cbar.set_label('Action', rotation=270, labelpad=20)
    cbar.ax.set_yticklabels(['Up', 'Down', 'Left', 'Right'])
    
    # Mark goal
    circle = plt.Circle(env.goal_pos, env.goal_radius, fill=False, 
                       edgecolor='white', linewidth=3, label='Goal')
    plt.gca().add_patch(circle)
    plt.plot(env.goal_pos[0], env.goal_pos[1], 'w*', markersize=20)
    
    plt.xlabel('x')
    plt.ylabel('y')
    plt.title('Learned Policy (Fitted Q-Learning)')
    plt.legend()
    plt.show()

visualize_policy(q_func)

## 7. Comparison with Discretization + Tabular Q-Learning

To understand the benefit of function approximation, let's compare with a naive discretization approach using tabular Q-Learning.

In [ ]:
def discretize_continuous_state(state, grid_size, n_bins):
    """Map continuous state to discrete bin index"""
    bin_size = grid_size / n_bins
    bin_x = int(np.clip(state[0] / bin_size, 0, n_bins - 1))
    bin_y = int(np.clip(state[1] / bin_size, 0, n_bins - 1))
    return bin_x * n_bins + bin_y

def q_iteration_tabular(dataset, grid_size, n_bins=20, gamma=0.95, num_iterations=20):
    """Q-Iteration with discretization (batch learning from same dataset)"""
    n_states = n_bins * n_bins
    n_actions = 4
    Q = np.zeros((n_states, n_actions))
    
    # Extract and discretize transitions
    transitions = []
    for state, action, reward, next_state, done in dataset:
        s_idx = discretize_continuous_state(state, grid_size, n_bins)
        s_next_idx = discretize_continuous_state(next_state, grid_size, n_bins)
        transitions.append((s_idx, action, reward, s_next_idx, done))
    
    # Q-Iteration updates
    for iteration in range(num_iterations):
        Q_old = Q.copy()
        
        for s_idx, action, reward, s_next_idx, done in transitions:
            if done:
                target = reward
            else:
                target = reward + gamma * np.max(Q_old[s_next_idx, :])
            
            Q[s_idx, action] = target
        
        if iteration % 5 == 0:
            mean_q = np.mean(Q[Q != 0])  # Mean of visited states
            print(f"Iteration {iteration}: mean Q-value = {mean_q:.3f}")
    
    return Q, n_bins

# Train tabular Fitted Q-Iteration using the SAME dataset as function approximation
print("Training tabular Q-Learning with discretization (using same dataset)...")
Q_tabular, n_bins = fitted_q_iteration_tabular(dataset, env.grid_size, n_bins=20, num_iterations=20)
print("Done!")

Create a policy from the tabular Q-function and evaluate it:

In [ ]:
def tabular_policy(state, Q_table, grid_size, n_bins):
    """Extract action from tabular Q-function"""
    # TODO: Compute state index and return argmax action
    raise NotImplementedError

# Evaluate tabular policy
tabular_pol = lambda s: tabular_policy(s, Q_tabular, env.grid_size, n_bins)
results_tabular = evaluate_policy(env, tabular_pol, num_episodes=50)

print("Tabular Q-Learning (discretized):")
print(f"  Mean score: {results_tabular['mean_score']:.2f} +/- {results_tabular['std_score']:.2f}")
print(f"  Success rate: {results_tabular['success_rate']:.1%}")

print("\nFitted Q-Learning (function approximation):")
print(f"  Mean score: {results_learned['mean_score']:.2f} +/- {results_learned['std_score']:.2f}")
print(f"  Success rate: {results_learned['success_rate']:.1%}")

Visualize the discretized policy:

In [ ]:
def visualize_discretized_policy(Q_table, n_bins, grid_size=5.0):
    """Visualize policy learned with discretization"""
    policy_map = np.zeros((n_bins, n_bins))
    
    for i in range(n_bins):
        for j in range(n_bins):
            state_idx = i * n_bins + j
            policy_map[i, j] = np.argmax(Q_table[state_idx, :])
    
    plt.figure(figsize=(8, 7))
    im = plt.imshow(policy_map, origin='lower', extent=[0, grid_size, 0, grid_size],
                    cmap='tab10', vmin=0, vmax=3, aspect='auto', interpolation='nearest')
    
    cbar = plt.colorbar(im, ticks=[0, 1, 2, 3])
    cbar.set_label('Action', rotation=270, labelpad=20)
    cbar.ax.set_yticklabels(['Up', 'Down', 'Left', 'Right'])
    
    # Mark goal
    circle = plt.Circle(env.goal_pos, env.goal_radius, fill=False,
                       edgecolor='white', linewidth=3, label='Goal')
    plt.gca().add_patch(circle)
    plt.plot(env.goal_pos[0], env.goal_pos[1], 'w*', markersize=20)
    
    # Draw grid lines to show discretization
    for i in range(n_bins + 1):
        val = i * grid_size / n_bins
        plt.axhline(val, color='gray', linewidth=0.5, alpha=0.5)
        plt.axvline(val, color='gray', linewidth=0.5, alpha=0.5)
    
    plt.xlabel('x')
    plt.ylabel('y')
    plt.title(f'Discretized Policy (Tabular Q-Learning, {n_bins}x{n_bins} bins)')
    plt.legend()
    plt.show()

visualize_discretized_policy(Q_tabular, n_bins)

## 8. Policy Gradient (REINFORCE)

Unlike value-based methods (Q-learning, Fitted Q-Iteration), **policy gradient methods** directly optimize a parametric policy $\pi_\theta(a|s)$ without explicitly learning a value function.

### 8.1 Softmax Policy Parameterization

We use a **softmax policy** with polynomial features:

$$\pi_\theta(a|s) = \frac{\exp(\theta_a^\top \phi(s))}{\sum_{a'=0}^{n_a-1} \exp(\theta_{a'}^\top \phi(s))}$$

where:
- $\theta = [\theta_0, \theta_1, \theta_2, \theta_3]$ are the parameters (one weight vector per action)
- $\phi(s)$ are polynomial features of the state (same as for Fitted Q-Iteration)
- The softmax ensures $\sum_a \pi_\theta(a|s) = 1$ and $\pi_\theta(a|s) \geq 0$

### 8.2 Policy Gradient Theorem

The gradient of the expected return $J(\theta) = \mathbb{E}_{\tau \sim \pi_\theta}[R(\tau)]$ is:

$$\nabla_\theta J(\theta) = \mathbb{E}_{\tau \sim \pi_\theta}\left[\sum_{t=0}^T \nabla_\theta \log \pi_\theta(a_t|s_t) \cdot R_t\right]$$

where $R_t = \sum_{t'=t}^T \gamma^{t'-t} r_{t'}$ is the **discounted return-to-go** from timestep $t$.

**Key insight**: We increase the probability of actions that led to high returns, and decrease the probability of actions that led to low returns.

### 8.3 Variance Reduction Techniques

**Baseline subtraction**: Instead of using $R_t$ directly, we use **advantages** $A_t = R_t - b$ where $b$ is a baseline (e.g., moving average of returns). This reduces variance without introducing bias.

**Advantage normalization**: We normalize advantages to have zero mean and unit variance within each episode. This stabilizes learning across different scales of rewards.

### 8.4 Bootstrap Learning

With sparse rewards, pure exploration can take very long to find successful trajectories. We **bootstrap** the policy by:
1. Collecting successful trajectories using a random policy
2. Using these trajectories to initialize the policy parameters
3. Then continuing with online learning

This provides an initial learning signal and speeds up convergence.

### 8.5 REINFORCE Algorithm

The REINFORCE algorithm:
1. Collect a trajectory by following $\pi_\theta$
2. Compute returns $R_t$ for each timestep
3. Compute advantages $A_t = R_t - \text{baseline}$
4. Update policy: $\theta \leftarrow \theta + \alpha \sum_t A_t \nabla_\theta \log \pi_\theta(a_t|s_t)$
5. Repeat

### 8.6 Exercise: Implement REINFORCE

Implement the REINFORCE algorithm with baseline and bootstrap learning.

**Implementation notes**:
- The gradient of $\log \pi_\theta(a|s)$ for softmax policy is: $\nabla_\theta \log \pi_\theta(a|s) = \phi(s) \cdot (\mathbf{1}_{a} - \pi_\theta(s))$
  - where $\mathbf{1}_a$ is a one-hot vector (1 for the selected action, 0 otherwise)
  - and $\pi_\theta(s)$ is the vector of action probabilities
- For bootstrapping, update policy parameters using successful trajectories before starting online learning

In [ ]:
def collect_initial_trajectories(env, n_episodes=50):
    """Collect successful trajectories using random policy to bootstrap learning"""
    trajectories = []
    for _ in range(n_episodes):
        states = []
        actions = []
        rewards = []
        
        state = env.reset()
        done = False
        
        while not done:
            action = np.random.randint(4)
            next_state, reward, done = env.step(action)
            
            states.append(state)
            actions.append(action)
            rewards.append(reward)
            
            state = next_state
        
        # Only keep successful trajectories (reward > 0 indicates success)
        if reward > 0:
            trajectories.append((states, actions, rewards))
    
    return trajectories

class SoftmaxPolicy:
    """Softmax policy with linear features"""
    def __init__(self, n_actions=4, state_degree=2):
        self.n_actions = n_actions
        self.poly = PolynomialFeatures(degree=state_degree)
        self.poly.fit(np.array([[0, 0]]))
        self.n_features = self.poly.transform(np.array([[0, 0]])).shape[1]
        
        self.theta = np.random.randn(n_actions, self.n_features) * 0.01
    
    def features(self, state):
        return self.poly.transform(state.reshape(1, -1)).flatten()
    
    def action_probabilities(self, state):
        phi = self.features(state)
        logits = self.theta @ phi
        # Numerical stability: subtract max
        logits = logits - np.max(logits)
        exp_logits = np.exp(logits)
        return exp_logits / np.sum(exp_logits)
    
    def sample_action(self, state):
        probs = self.action_probabilities(state)
        return np.random.choice(self.n_actions, p=probs)
    
    def greedy_action(self, state):
        probs = self.action_probabilities(state)
        return np.argmax(probs)

def compute_returns(rewards, gamma=0.95):
    returns = np.zeros(len(rewards))
    running_return = 0
    for t in reversed(range(len(rewards))):
        running_return = rewards[t] + gamma * running_return
        returns[t] = running_return
    return returns

def reinforce(env, n_actions=4, state_degree=2, 
             gamma=0.95, lr=0.001, num_episodes=500, initial_trajectories=None):
    """REINFORCE algorithm: online policy gradient with baseline"""
    policy = SoftmaxPolicy(n_actions, state_degree)
    episode_returns = []
    
    # Bootstrap with initial successful trajectories if provided
    if initial_trajectories:
        print(f"Bootstrapping with {len(initial_trajectories)} successful trajectories...")
        for states, actions, rewards in initial_trajectories:
            returns = compute_returns(rewards, gamma)
            
            for t in range(len(states)):
                state = states[t]
                action = actions[t]
                R_t = returns[t]
                
                phi = policy.features(state)
                probs = policy.action_probabilities(state)
                
                # TODO Exercise 5a: Compute gradient of log pi(a|s)
                # Hint: For the action taken, grad = phi * (1 - prob[action])
                #       For other actions, grad = -phi * prob[action]
                grad_log_pi = np.zeros_like(policy.theta)
                raise NotImplementedError
                
                policy.theta += lr * R_t * grad_log_pi
    
    for episode in range(num_episodes):
        # Collect trajectory using current policy
        states = []
        actions = []
        rewards = []
        
        state = env.reset()
        done = False
        
        while not done:
            action = policy.sample_action(state)
            next_state, reward, done = env.step(action)
            
            states.append(state)
            actions.append(action)
            rewards.append(reward)
            
            state = next_state
        
        # Compute returns
        returns = compute_returns(rewards, gamma)
        episode_returns.append(sum(rewards))
        
        # TODO Exercise 5b: Compute baseline and advantages
        # Baseline: mean of last 50 episode returns (or 0 if < 10 episodes)
        # Advantages: returns - baseline, then normalize if len > 1
        baseline = None
        advantages = None
        raise NotImplementedError
        
        # TODO Exercise 5c: Update policy using policy gradient
        # For each timestep, compute grad_log_pi (same as bootstrap above)
        # and update: policy.theta += lr * advantage[t] * grad_log_pi
        raise NotImplementedError
        
        if episode % 100 == 0 and episode > 0:
            avg_return = np.mean(episode_returns[-100:])
            print(f"Episode {episode}: avg return (last 100) = {avg_return:.3f}")
    
    return policy

Collect initial successful trajectories and train the policy gradient method:

In [ ]:
# Collect initial successful trajectories using random policy
initial_trajs = collect_initial_trajectories(env, n_episodes=50)
print(f"Collected {len(initial_trajs)} successful trajectories")

# Train REINFORCE with bootstrapping
pg_policy = reinforce(env, state_degree=2, lr=0.001, num_episodes=500, initial_trajectories=initial_trajs)

Evaluate the policy gradient policy:

In [ ]:
# Evaluate policy gradient policy
pg_greedy = lambda s: pg_policy.greedy_action(s)
results_pg = evaluate_policy(env, pg_greedy, num_episodes=50, render_episodes=3)

print("Policy Gradient:")
print(f"  Mean score: {results_pg['mean_score']:.2f} +/- {results_pg['std_score']:.2f}")
print(f"  Success rate: {results_pg['success_rate']:.1%}")

print("\nComparison:")
print(f"  Fitted Q-Learning:  {results_learned['success_rate']:.1%} success")
print(f"  Tabular (discrete): {results_tabular['success_rate']:.1%} success")
print(f"  Policy Gradient:    {results_pg['success_rate']:.1%} success")
print(f"  Random:             {results_random['success_rate']:.1%} success")

Visualize policy gradient trajectories:

In [ ]:
env.render(results_pg['trajectories'])

Visualize the policy gradient policy:

In [ ]:
def visualize_pg_policy(policy, grid_size=5.0, resolution=100):
    """Visualize policy gradient policy as heatmap"""
    x = np.linspace(0, grid_size, resolution)
    y = np.linspace(0, grid_size, resolution)
    X, Y = np.meshgrid(x, y)
    
    policy_map = np.zeros((resolution, resolution))
    for i in range(resolution):
        for j in range(resolution):
            state = np.array([X[i, j], Y[i, j]])
            policy_map[i, j] = policy.greedy_action(state)
    
    plt.figure(figsize=(8, 7))
    im = plt.imshow(policy_map, origin='lower', extent=[0, grid_size, 0, grid_size],
                    cmap='tab10', vmin=0, vmax=3, aspect='auto')
    
    cbar = plt.colorbar(im, ticks=[0, 1, 2, 3])
    cbar.set_label('Action', rotation=270, labelpad=20)
    cbar.ax.set_yticklabels(['Up', 'Down', 'Left', 'Right'])
    
    circle = plt.Circle(env.goal_pos, env.goal_radius, fill=False,
                       edgecolor='white', linewidth=3, label='Goal')
    plt.gca().add_patch(circle)
    plt.plot(env.goal_pos[0], env.goal_pos[1], 'w*', markersize=20)
    
    plt.xlabel('x')
    plt.ylabel('y')
    plt.title('Learned Policy (Policy Gradient)')
    plt.legend()
    plt.show()

visualize_pg_policy(pg_policy)